In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from logger import logger
import shutil, os, re

In [ ]:
csv_results_dir = '../files/csv/'


scrap_old_flag = 0

if scrap_old_flag==1:
    shutil.rmtree(csv_results_dir, ignore_errors=True)
    os.mkdir(csv_results_dir)

In [ ]:
# url = "https://www.amazon.com/gp/bestsellers/books"
# HEADERS = ({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15', 'Accept-Language' : 'en-CA,en-US;q=0.9,en;q=0.8' })

In [ ]:
# page = requests.get(url, headers = HEADERS)
# soup = BeautifulSoup(page.text, 'html' )

In [ ]:
# books = soup.find_all("div", attrs = {'a-column a-span12 a-text-center _cDEzb_grid-column_2hIsc'})

In [ ]:
# book = books[11]
# book

In [ ]:
# for book in books:
#     book_values = {
#         "rank" : get_rank(book),
#         "asin" : get_asin(book),
#         "name": get_name(book),
#         "author" : get_author(book),
#         "type_case" :get_type_case(book),
#         "price" : get_price(book),
#         "link" : get_link(book)
#     }
    
#     ranking_df = pd.concat([ranking_df, pd.DataFrame.from_dict([book_values])], ignore_index=True)
# ranking_df

In [ ]:
def get_rank(book_div):
    try:
        rank = book_div.find("span", attrs ="zg-bdg-text").text
        return rank
    except:
        return ""
    
    
def get_asin(book_div):
    try:
        asin = book_div.find("a", attrs="a-link-normal a-text-normal").get("href").split("ref")[0].split("dp")[-1].replace("/","")
        return asin
    except:
        return ""
    
def get_name(book_div):
    try:
        name = book_div.find("img").get("alt")
        # "name" : book.find("div", attrs='_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y').text,
        return name
    except:
        return ""
    
def get_author(book_div):
    try:
        author = book_div.find_all("div", attrs='_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y')[-1].text
        return author
    except:
        return ""
    
def get_type_case(book_div):
    try:
        type_case = book_find("span", attrs= "a-size-small a-color-secondary a-text-normal").text
        return type_case
    except:
        return ""
    
def get_price(book_div):
    try:
        price = book_div.find('span', attrs= "_cDEzb_p13n-sc-price_3mJ9Z").text
        return price
    except:
        return ""
    
def get_link(book_div):
    try:
        link = "https://amazon.com"+ book_div.find("a", attrs="a-link-normal a-text-normal").get("href").split("ref")[0]
        return link
    except:
        return ""
    
def get_category(soup):
    try:
        category = soup.find("title").text.split(":")[1].strip().replace("Best ","")[:30]
        return category
    except:
        return ""


In [23]:
HEADERS = ({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15', 'Accept-Language' : 'en-CA,en-US;q=0.9,en;q=0.8' })
# writer = pd.ExcelWriter("Ranks.xlsx")

for i in [8, 11, 12, 13, 14]:
    
    ranking_df = pd.DataFrame()
    url = f"https://www.amazon.com/gp/bestsellers/books/{str(i)}/ref=pd_zg_hrsr_books"
    #print(url)
    
    page = requests.get(url, headers = HEADERS)
    
    if(str(page) == "<Response [200]>"):
        soup = BeautifulSoup(page.text, 'html')
        category = get_category(soup)
        books = soup.find_all("div", attrs = {'a-column a-span12 a-text-center _cDEzb_grid-column_2hIsc'})
        
        print(f"Processing page {i} - Category {category} with {len(books)} books")
        logger(f"Processing page {i} - Category {category} with {len(books)} books", logfile="log.txt")
        
        if len(books)>0:
            for book in books:
                book_values = {
                    "category": category,
                    "rank" : get_rank(book),
                    "asin" : get_asin(book),
                    "name": get_name(book),
                    "author" : get_author(book),
                    "type_case" :get_type_case(book),
                    "price" : get_price(book),
                    "link" : get_link(book)
                }

                ranking_df = pd.concat([ranking_df, pd.DataFrame.from_dict([book_values])], ignore_index=True)
            ranking_df.to_csv(csv_results_dir+f"{i} - {category}.csv", index=False)
        else:
            print("")
            logger(f"PAGE {i} - {url} \t\t ----------------ERROR - NO BOOKS found", logfile="log.txt")
            logger(f'{i}, ', logfile = "error_list.txt")
    else:
        print(f"PAGE {i} - {url} \t\t ----------------ERROR - reading page")
        logger(f"PAGE {i} - {url} \t\t ----------------ERROR - reading page", logfile="log.txt")
        logger(f'{i}, ', logfile = "error_list.txt")
    time.sleep(5)
# writer.close()

Processing page 8 - Category undefined with 0 books

Processing page 11 - Category undefined with 0 books

PAGE 12 - https://www.amazon.com/gp/bestsellers/books/12/ref=pd_zg_hrsr_books 		 ----------------ERROR - reading page
PAGE 13 - https://www.amazon.com/gp/bestsellers/books/13/ref=pd_zg_hrsr_books 		 ----------------ERROR - reading page
Processing page 14 - Category undefined with 0 books

